In [ ]:
import hashlib
import datetime
from flask import Flask, jsonify, request, render_template

In [ ]:
class Block:
    def __init__(self, index, timestamp, data, previous_hash):
        self.index = index
        self.timestamp = timestamp
        self.data = data
        self.previous_hash = previous_hash
        self.hash = self.calculate_hash()

    def calculate_hash(self):
        block_string = f"{self.index}{self.timestamp}{self.data}{self.previous_hash}"
        return hashlib.sha256(block_string.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = [self.create_genesis_block()]

    def create_genesis_block(self):
        return Block(0, str(datetime.datetime.now()), "Genesis Block", "0")

    def get_latest_block(self):
        return self.chain[-1]

    def add_block(self, new_data):
        latest_block = self.get_latest_block()
        new_block = Block(len(self.chain), str(datetime.datetime.now()), new_data, latest_block.hash)
        self.chain.append(new_block)

    def is_chain_valid(self):
        for i in range(1, len(self.chain)):
            current_block = self.chain[i]
            previous_block = self.chain[i - 1]

            if current_block.hash != current_block.calculate_hash():
                return False

            if current_block.previous_hash != previous_block.hash:
                return False

        return True

# Initialize Flask App and Blockchain
app = Flask(__name__)
blockchain = Blockchain()

@app.route("/")
def index():
    return render_template("index.html")

@app.route("/add_block", methods=["POST"])
def add_block():
    data = request.form["data"]
    if data.strip():
        blockchain.add_block(data)
        return jsonify({"message": "Block added successfully!"}), 200
    else:
        return jsonify({"message": "Please enter valid data."}), 400

@app.route("/get_chain", methods=["GET"])
def get_chain():
    chain_data = [
        {
            "index": block.index,
            "timestamp": block.timestamp,
            "data": block.data,
            "previous_hash": block.previous_hash,
            "hash": block.hash,
        }
        for block in blockchain.chain
    ]
    return jsonify({"chain": chain_data, "length": len(chain_data)}), 200
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Blockchain Transactions</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            text-align: center;
            padding: 20px;
        }
        textarea {
            width: 80%;
            height: 300px;
        }
        input[type="text"] {
            width: 60%;
            padding: 10px;
        }
        button {
            padding: 10px 20px;
            margin: 5px;
        }
    </style>
</head>
<body>
    <h1>Blockchain Transactions</h1>
    <h2>TCS 407 Lab Work 1 by Group 2</h2>
    <form id="add-block-form">
        <input type="text" name="data" placeholder="Enter data for new block" required>
        <button type="submit">Add Block</button>
    </form>
    <button onclick="fetchChain()">View Blockchain</button>
    <button onclick="verifyChain()">Verify Blockchain</button>
    <textarea id="blockchain-display" readonly></textarea>

    <script>
        async function fetchChain() {
            const response = await fetch("/get_chain");
            const data = await response.json();
            const chain = data.chain.map(block => `
                Index: ${block.index}
                Timestamp: ${block.timestamp}
                Data: ${block.data}
                Previous Hash: ${block.previous_hash}
                Hash: ${block.hash}
                -----------------------------
            `).join("\\n");
            document.getElementById("blockchain-display").value = chain;
        }

        async function verifyChain() {
            const response = await fetch("/verify_chain");
            const data = await response.json();
            alert(data.message);
        }

        document.getElementById("add-block-form").addEventListener("submit", async (event) => {
            event.preventDefault();
            const formData = new FormData(event.target);
            const response = await fetch("/add_block", {
                method: "POST",
                body: formData
            });
            const data = await response.json();
            alert(data.message);
            event.target.reset();
        });
    </script>
</body>
</html>

In [ ]:
if __name__ == "__main__":
    app.run(port=5000, debug=True)